In [7]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [9]:
red = pd.read_csv("/Users/ellalee/Downloads/wine+quality/winequality-red.csv", sep = ';')
white = pd.read_csv("/Users/ellalee/Downloads/wine+quality/winequality-white.csv", sep = ';')

red['color'] = 'red'
white['color'] = 'white'

df = pd.concat([red, white], ignore_index=True)

both = df
df_array = [red, white, both]
results = []

def categorize_quality(quality):
    if quality < 7:
        return 'low'
    else:
        return 'high'

df['quality_category'] = df['quality'].apply(categorize_quality)
red['quality_category'] = red['quality'].apply(categorize_quality)
white['quality_category'] = white['quality'].apply(categorize_quality)

df_array = [red, white, df]
wine_names = ['red', 'white', 'red and white']

In [15]:
results=[]

for df in df_array:
    X = df.drop(['quality', 'quality_category', 'color'], axis=1)
    y = df['quality_category']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    results.append(report)

for name, report in zip(wine_names, results):
    print("Decision tree for " + name + " dataset")
    print(f"accuracy: {report['accuracy']}")
    print("----------------------")

Decision tree for red dataset
accuracy: 0.871875
----------------------
Decision tree for white dataset
accuracy: 0.8295918367346938
----------------------
Decision tree for red and white dataset
accuracy: 0.8346153846153846
----------------------
